In [ ]:
# Simulation parameters for modified roulette
n_players = 10
n_spins = 1000
starting_wealth = 1000
bet_size = 10

# Generate wealth paths for two groups of players
np.random.seed(42)

# Group 1: Novice players who bet randomly
novice_wealths = np.zeros((n_players, n_spins + 1))
novice_wealths[:, 0] = starting_wealth

# Group 2: Pro players who bet optimally based on previous outcome
pro_wealths = np.zeros((n_players, n_spins + 1))
pro_wealths[:, 0] = starting_wealth

# Initial state (50-50 chance for first spin)
prev_red = np.random.random() < 0.5

for i in range(n_spins):
    # Determine outcome based on previous state
    p_red = 0.55 if prev_red else 0.45
    is_red = np.random.random() < p_red
    prev_red = is_red
    
    # Novice players bet randomly
    novice_bets = np.random.random(n_players) < 0.5  # True = bet red, False = bet black
    results_novice = novice_bets == is_red  # Win if correctly predicted
    wins_novice = results_novice * bet_size
    losses_novice = ~results_novice * bet_size
    net_results_novice = wins_novice - losses_novice
    novice_wealths[:, i+1] = novice_wealths[:, i] + net_results_novice
    
    # Pro players bet optimally (bet on more likely outcome based on previous)
    # If previous was red, p(red)=0.55 so bet red, if black p(black)=0.55 so bet black
    pro_bets = np.full(n_players, prev_red)  # Bet red if prev was red, black if prev was black
    
    # Each pro player has 55% chance of winning when betting optimally
    results_pro = np.random.random(n_players) < 0.55  # Simulate 55% win rate
    wins_pro = results_pro * bet_size
    losses_pro = ~results_pro * bet_size
    net_results_pro = wins_pro - losses_pro
    pro_wealths[:, i+1] = pro_wealths[:, i] + net_results_pro

# Create figure with subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Novice Players (Random Bets)', 'Pro Players (Bet Based on Previous Outcome)')
)

# Add traces for Novice players
x_vals = np.arange(n_spins + 1)
for i in range(n_players):
    opacity = 0.3 + (0.7 * i/n_players)
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=novice_wealths[i],
            mode='lines',
            line=dict(color=f'rgba(255, 0, 0, {opacity})', width=2),
            showlegend=False
        ),
        row=1, col=1
    )

# Add Novice players trendline
novice_mean = np.mean(novice_wealths, axis=0)
z_novice = np.polyfit(x_vals, novice_mean, 1)
p_novice = np.poly1d(z_novice)
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=p_novice(x_vals),
        mode='lines',
        line=dict(color='white', width=2, dash='dash'),
        name='Trend',
        showlegend=False
    ),
    row=1, col=1
)

# Add traces for Pro players
for i in range(n_players):
    opacity = 0.3 + (0.7 * i/n_players)
    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=pro_wealths[i],
            mode='lines',
            line=dict(color=f'rgba(0, 255, 0, {opacity})', width=2),
            showlegend=False
        ),
        row=1, col=2
    )

# Add Pro players trendline
pro_mean = np.mean(pro_wealths, axis=0)
z_pro = np.polyfit(x_vals, pro_mean, 1)
p_pro = np.poly1d(z_pro)
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=p_pro(x_vals),
        mode='lines',
        line=dict(color='white', width=2, dash='dash'),
        name='Trend',
        showlegend=False
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    height=500,
    width=1300,
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white')
)

# Update axes
for i in range(1, 3):
    fig.update_xaxes(
        title='Number of Spins',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )
    fig.update_yaxes(
        title='Wealth ($)',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )

fig.show()